# VGG16 with Gabor filters

In [ ]:
from keras.applications.vgg16 import VGG16
weights = None  # 'imagenet'
input_shape = (224, 224, 3)  # (224, 224, 16)
classes = 10
model = VGG16(include_top=True, weights=weights, input_tensor=None, input_shape=input_shape, pooling=None, classes=classes)

In [ ]:
model.summary()

In [ ]:
from keras.layers import InputLayer
model.layers.pop(0)
gabor_input = InputLayer(input_shape=(224, 224, 32))
gabor_output = model(gabor_input)
gabor_model = Model(gabor_input, gabor_output)

In [ ]:
lambdas = [3, 4, 5, 6, 7, 8]  # 3 <= lambd <= W/2
sigmas = [1, 2, 3] # 4]  
bandwidths = np.linspace(0.4, 2.6, num=3)  # ~1.5 <= bw <= ~3

n_thetas = 8
n_psis = 4  # 1, 2, 4
n_gammas = 2

gabor = {'sigmas': [sigma],
         'lambdas': [lambd],
         'thetas': np.linspace(0, 2*np.pi, n_thetas, endpoint=False),        
         'psis': np.linspace(0, 2*np.pi, n_psis, endpoint=False),
         'gammas': np.linspace(1, 0, n_gammas, endpoint=False),
         'ksize': (31, 31),
        }

In [ ]:
conda install opencv

In [ ]:
import cv2

# Run GaborNet

In [ ]:
import os
import numpy as np

def calc_bandwidth(lambd, sigma):
    r = np.pi*sigma/lambd
    c = np.sqrt(np.log(2)/2)
    return np.log2((r + c)/(r - c))

def calc_sigma(lambd, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return lambd * c / np.pi  * (p + 1) / (p - 1)

def calc_lambda(sigma, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return sigma * np.pi / c  * (p - 1) / (p + 1)

# Set parameters
project_root = os.path.realpath(os.pardir)
data_set = 'pixel'
# stimulus_sets = ['static', 'jitter']
stimulus_set = 'jitter'
start_trial = 1
num_trials = 5
# lambdas = [3, 4, 5, 6, 7, 8]
# lambdas = [4, 8, 16]
# sigmas = [2, 4, 8]
# bandwidths = np.linspace(1, 1.8, num=3)

# sigmas = [1, 2, 3, 4, 5]
# sigmas = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.1, 2.2, 2.3, 2.4, 2.5]
# sigmas = [1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7, 2.9]
# sigmas = [1.2, 1.4, 1.6, 1.8, 2.2, 2.4, 2.6, 2.8]
sigmas = [1.5, 2.5, 3.5, 4.5]
bandwidths = np.linspace(1, 1.8, num=5)

epochs = 20
save_loss = 0
data_augmentation = 0
fresh_data = 0
n_gpus = 1

In [ ]:
import os
from tqdm import tqdm_notebook

# for stimulus_set in tqdm_notebook(stimulus_sets, desc="Set"):
# for bandwidth in tqdm_notebook(bandwidths, desc='$b$', leave=True):
for trial in tqdm_notebook(range(start_trial, start_trial+num_trials), desc='Trial'):
    for sigma in tqdm_notebook(sigmas, desc='$\sigma$', leave=True):
        for bandwidth in tqdm_notebook(bandwidths, desc='$b$', leave=False):
#         for lambd in tqdm_notebook(lambdas, desc='$\lambda$', leave=True):
#             sigma = calc_sigma(lambd, bandwidth)
            lambd = calc_lambda(sigma, bandwidth)
            trial_label = f"{trial}_sigma={float(sigma):.2}_lambd={float(lambd):.2}"
            args = (f"--data_set {data_set} --stimulus_set {stimulus_set} "
                    f"--trial_label {trial_label} --sigma {sigma} --lambd {lambd} "
                    f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                    f"--n_gpus {n_gpus} --epochs {epochs} --save_loss {save_loss}")  #  --model_name {model_name}
            os.system(f'python3 gabornet.py {args}')

In [ ]:
calc_lambda(1.5, 1)

In [ ]:
len('')

In [ ]:
import os
import subprocess
from tqdm import tqdm_notebook

# Set parameters
project_root = os.path.realpath(os.pardir)
filter_type = 'dog'
data_set = 'pixel'
stimulus_set = 'jitter'
start_trial = 1
num_trials = 5

sigmas = [1, 2, 3, 4, 5]
# sigmas = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.1, 2.2, 2.3, 2.4, 2.5]
# sigmas = [1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7, 2.9]
# sigmas = [1.2, 1.4, 1.6, 1.8, 2.2, 2.4, 2.6, 2.8]
# bandwidths = np.linspace(1, 1.8, num=5)
r_sigmas = [1.1, 1.2, 1.5, 2, 2.5, 3]

epochs = 20
save_loss = 0
data_augmentation = 0
fresh_data = 0
n_gpus = 1

for trial in tqdm_notebook(range(start_trial, start_trial+num_trials), desc='Trial'):
    for sigma in tqdm_notebook(sigmas, desc='$\sigma$', leave=True):
        for r_sigma in tqdm_notebook(r_sigmas, desc='$r_\sigma$', leave=False):
            trial_label = f"{trial}_sigma={float(sigma):.2}_r_sigma={float(r_sigma):.2}"
            args = (f"--filter_type {filter_type} --data_set {data_set} --stimulus_set {stimulus_set} "
                    f"--trial_label {trial_label} --sigma {sigma} --r_sigma {r_sigma} "
                    f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                    f"--n_gpus {n_gpus} --epochs {epochs} --save_loss {save_loss}")
#             os.system(f'python3 gabornet.py {args}')
            subprocess.call(f'python3 gabornet.py {args}')

## DoGnet

In [ ]:
!git pull

In [ ]:
import subprocess
trial_label = "TEST"
args = (f"--filter_type {filter_type} --data_set {data_set} --stimulus_set {stimulus_set} "
                    f"--trial_label {trial_label} --sigma {sigma} --r_sigma {r_sigma} "
                    f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                    f"--n_gpus {n_gpus} --epochs {epochs} --save_loss {save_loss}")
os.system(f'python3 gabornet.py {args}')

In [ ]:
import os
from tqdm import tqdm_notebook

# Set parameters
project_root = os.path.realpath(os.pardir)
filter_type = 'dog'
data_set = 'pixel'
stimulus_set = 'jitter'
start_trial = 1
num_trials = 5

sigmas = [1, 2, 3, 4, 5]
# r_sigmas = [1.1, 1.2, 1.5, 2, 2.5, 3]
r_sigmas = [1.3, 1.4, 1.6, 1.7, 1.8, 1.9]

epochs = 20
save_loss = 0
data_augmentation = 0
fresh_data = 0
n_gpus = 1

for trial in tqdm_notebook(range(start_trial, start_trial+num_trials), desc='Trial'):
    for sigma in tqdm_notebook(sigmas, desc='$\sigma$', leave=True):
        for r_sigma in tqdm_notebook(r_sigmas, desc='$r_\sigma$', leave=False):
            trial_label = f"{trial}_sigma={float(sigma):.2}_r_sigma={float(r_sigma):.2}"
            args = (f"--filter_type {filter_type} --data_set {data_set} --stimulus_set {stimulus_set} "
                    f"--trial_label {trial_label} --sigma {sigma} --r_sigma {r_sigma} "
                    f"--data_augmentation {data_augmentation} --fresh_data {fresh_data} "
                    f"--n_gpus {n_gpus} --epochs {epochs} --save_loss {save_loss}")
            os.system(f'python3 gabornet.py {args}')

# Plot performance

In [ ]:
# Load accuracy scores and plot
%matplotlib inline
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


# data_set = 'pixel'
stimulus_set = 'jitter'  # 'static'  # 'jitter'
noise_types = ['Original', 'Salt-and-pepper', 'Additive', 'Single-pixel']  # 'Original'
test_conditions = ['Same', 'Diff', 'NoPix']
results_dir = os.path.join(project_root, 'results', data_set, stimulus_set)

rows = []
test_rows = []

# for stimulus_set in stimulus_sets:
for trial in range(1, 1+num_trials):
    for noise_type in noise_types:
        for lambd in lambdas:

            trial_label = f"{trial}_lambd={lambd}"
            model_name = f"{noise_type}_{trial_label}"

            acc_scores = np.load(os.path.join(results_dir, f'{model_name}_ACC.npy'))
            valacc_scores = np.load(os.path.join(results_dir, f'{model_name}_VALACC.npy'))
            loss = np.load(os.path.join(results_dir, f'{model_name}_LOSS.npy'))
            valloss = np.load(os.path.join(results_dir, f'{model_name}_VALLOSS.npy'))

            with open(os.path.join(results_dir, f'{model_name}_CONDVALACC.json'), "r") as jf:
                cond_acc = json.load(jf)
            with open(os.path.join(results_dir, f'{model_name}_CONDVALLOSS.json'), "r") as jf:
                cond_loss = json.load(jf)

            for condition in test_conditions:
                test_rows.append({'Trial': trial, 'Noise Type': noise_type,
                                 'Condition': condition, 'Lambda': lambd,
                                  'Loss': cond_loss[condition], 'Accuracy': cond_acc[condition]})
            for epoch in range(epochs):
                rows.append({'Trial': trial, 'Noise Type': noise_type, 'Lambda': lambd, 
                             'Evaluation': 'Testing', 'Epoch': epoch+1, 'Loss': valloss[epoch], 
                             'Accuracy': valacc_scores[epoch]})

                rows.append({'Trial': trial, 'Noise Type': noise_type, 'Lambda': lambd, 
                             'Evaluation': 'Training', 'Epoch': epoch+1, 'Loss': loss[epoch], 
                             'Accuracy': acc_scores[epoch]})

scores = pd.DataFrame(rows, columns=['Trial', 'Noise Type', 'Evaluation', 'Lambda', 'Epoch', 'Loss', 'Accuracy'])
test_scores = pd.DataFrame(test_rows, columns=['Trial', 'Noise Type', 'Condition', 'Lambda', 'Loss', 'Accuracy'])
# scores
# test_scores

In [ ]:
test_scores

In [ ]:
# fig = plt.figure()
# g = sns.relplot(x='Epoch', y='Accuracy', style='Retina Out Width', hue='Evaluation', row='VVS Layers', col='Noise Type', kind='line', data=scores)
g = sns.relplot(x='Epoch', y='Accuracy', hue='Evaluation', row='Lambda', col='Noise Type', kind='line', data=scores)
# g = sns.relplot(x='Epoch', y='Loss', style='Retina Out Width', hue='Evaluation', row='VVS Layers', col='Noise Type', kind='line', data=scores)

In [ ]:
fig = plt.figure(figsize=(16,6))
# g = sns.catplot(x="Condition", y="Accuracy", hue="Retina Out Width", row="VVS Layers", col="Noise Type", kind="bar", data=test_scores)
g = sns.catplot(x="Condition", y="Accuracy", row='Lambda', col="Noise Type", kind="bar", data=test_scores)
# g = sns.catplot(x="Condition", y="Loss", hue="Retina Out Width", row="VVS Layers", col="Noise Type", kind="bar", data=test_scores)

In [ ]:
# from keras.layers import InputLayer
model.layers.pop(1)  # First convolutional layer
gabor_input = InputLayer(input_shape=(224, 224, 32))
gabor_output = model(gabor_input)
gabor_model = Model(gabor_input, gabor_output)

In [ ]:
from matplotlib import pyplot as plt
# import cv2
# from keras import backend as K
# import numpy as np

image_path = "/workspace/data/Lenna.png"
# img = image.load_img(image_path)
img = plt.imread(image_path)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)

gabor_filter(img, kernel_tensor=gft)

In [ ]:
import tensorflow as tf
with tf.Session().as_default():
    plt.imshow(gabor_filter(img, kernel_tensor=gft).eval()[0,:,:,-1])

In [ ]:
type(gabor_filters)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='vgg.png')